In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense ,LSTM
import matplotlib.pyplot as plt
import math
from numpy.random import seed
seed(3)
import tensorflow as tf
tf.compat.v1.set_random_seed(2)

In [11]:
Epochs=np.linspace(25,35,11,dtype=int)
Nodes=np.linspace(25,35,11,dtype=int)
Batch=5
no=9

In [12]:
df1=pd.read_csv("/content/drive/MyDrive/States/NEW_MODEL/COVID-19 Cases(27-10-2021).csv")
p="/content/drive/MyDrive/States/NEW_MODEL/INDIA"
df=df1[df1["Region"]=="India"]

In [13]:
df

,S. No.,Date,Region,Confirmed Cases,Active Cases,Cured/Discharged,Death
0,1,01/01/1970,India,26766298,323025.0,26138541.0,304732.0
1,2,12/03/2020,India,74,71.0,3.0,0.0
2,3,13/03/2020,India,75,71.0,3.0,1.0
3,4,14/03/2020,India,84,72.0,10.0,2.0
4,5,15/03/2020,India,107,95.0,10.0,2.0
...,...,...,...,...,...,...,...
575,576,22/10/2021,India,34143236,175745.0,33514449.0,453042.0
576,577,23/10/2021,India,34159562,173728.0,33532126.0,453708.0
577,578,24/10/2021,India,34175468,172594.0,33548605.0,454269.0
578,579,25/10/2021,India,34188774,166695.0,33567367.0,454712.0


In [14]:
data=df.filter(["Active Cases"])
dataset=data[1:].values
training_data_len=math.ceil(len(dataset)*.9)

In [15]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
l=150

train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]

for i in range(l,len(train_data)):
  x_train.append(train_data[i-l:i,0])
  y_train.append(train_data[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(len(x_train),len(x_train[0]),1))



In [16]:
for N in Nodes:
  for E in Epochs:
    if N!=0:
      model=Sequential()
      model.add(LSTM(N,return_sequences=False,input_shape=(len(x_train[0]),1)))
      # model.add(LSTM(10,return_sequences=False))
      model.add(Dense(12))
      model.add(Dense(1))
      model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy']) #mean_squared_error
      hist=model.fit(x_train,y_train,batch_size=Batch,epochs=E) #upto 50 epoch

      loss=hist.history["loss"]

      Testsss=p+"/Test/"+str(N)+"/"+str(E)+".xlsx"
      lossss=p+"/Train/"+str(N)+"/"+str(E)+".xlsx"
      Predsss=p+"/Pred/"+str(N)+"/"+str(E)+".xlsx"
      dict1={"Epochs":np.linspace(1,E,E),"Loss":loss}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(lossss)
      df_3.to_excel(xl)
      xl.save()

      #test
      test_data=scaled_data[training_data_len-l:,:]
      x_test=[]
      y_test=dataset[training_data_len:,:]

      for i in range(l,len(test_data)):
        x_test.append(test_data[i-l:i,0])

      #convert data to numpy
      x_test=np.array(x_test)

      #reshape
      x_test=np.reshape(x_test,(len(x_test),len(x_test[0]),1))

      predictions=model.predict(x_test)
      Test_Accuracy=model.evaluate(x_test,predictions)
      predictions=scaler.inverse_transform(predictions)


      predictions1=np.reshape(predictions,(len(predictions)))

      dict1={"Days":np.linspace(len(x_train)+1,len(x_train)+len(x_test),len(x_test)),"Test Prediction":predictions1}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(Testsss)
      df_3.to_excel(xl)
      xl.save()

      xx=np.reshape(x_test[-1],(1,l,1))
      prediction=model.predict(xx)
      pred=np.array([])
      for i in range(300):
        prediction=model.predict(xx)
        if prediction[0]<0:
          prediction[0]=xx[0][-1]
        pred=np.append(pred,prediction[0],axis=0)
        xx=np.delete(xx,0,axis=1)
        xx=np.append(xx[0],prediction,axis=0)
        xx=np.reshape(xx,(1,l,1))

      

      pred=np.reshape(pred,(len(pred),1))
      pred=scaler.inverse_transform(pred)

      XX=np.linspace(len(dataset),len(dataset)+pred.size-1,pred.size)

      pred1=np.reshape(pred,(len(pred)))
      dict1={"Days":XX,"Third Wave":pred1}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(Predsss)
      df_3.to_excel(xl)
      xl.save()
      print(N,E)


    

            


Epoch 1/25
75/75 [==============================] - 5s 42ms/step - loss: 0.0139 - accuracy: 0.0000e+00
Epoch 2/25
75/75 [==============================] - 3s 42ms/step - loss: 0.0018 - accuracy: 0.0027
Epoch 3/25
75/75 [==============================] - 3s 41ms/step - loss: 4.1444e-04 - accuracy: 0.0027
Epoch 4/25
75/75 [==============================] - 3s 41ms/step - loss: 2.5112e-04 - accuracy: 0.0027
Epoch 5/25
75/75 [==============================] - 3s 42ms/step - loss: 1.9309e-04 - accuracy: 0.0027
Epoch 6/25
75/75 [==============================] - 3s 42ms/step - loss: 1.7386e-04 - accuracy: 0.0027
Epoch 7/25
75/75 [==============================] - 3s 42ms/step - loss: 1.6503e-04 - accuracy: 0.0027
Epoch 8/25
75/75 [==============================] - 3s 42ms/step - loss: 1.7935e-04 - accuracy: 0.0027
Epoch 9/25
75/75 [==============================] - 3s 42ms/step - loss: 1.5404e-04 - accuracy: 0.0027
Epoch 10/25
75/75 [==============================] - 3s 42ms/step - loss: 1.4

FileNotFoundError: ignored

In [ ]:
plt.plot(pred1)

In [ ]:
xx[0]